In [ ]:
#load "../Utils.fsx"
open Utils

let input = File.ReadAllLines "input.txt"
let template = input[0] |> Seq.toList
let rules = input[2..] |> Seq.map(fun s -> (s[0],s[1]), s[6]) |> Map.ofSeq

let rec expand (rules: Map<char*char, char>) = function
  | a :: (b :: _ as rest) -> a :: rules[a,b] :: expand rules rest
  | [c] -> [c]

let afterStep n f s = generate f s |> Seq.item n

let maxMin s = (Seq.max s, Seq.min s)

let mergeCounts =
  Map.fold (fun m k v -> m |> Map.add k (v + (Map.tryFind k m |> Option.defaultValue 0L)))

let charCounts (rules: Map<char*char, char>) s i = 
  let rec count = memoize <| function
    | (c,d,0) -> Map.empty
    | (c,d,1) -> Map.ofList [rules[c,d], 1L]
    | (c,d,n) as arg -> [Map [rules[c,d], 1L]; count (c, rules[c,d], n-1); count (rules[c,d], d, n-1)] |> Seq.reduce mergeCounts
  
  s |> Seq.pairwise |> Seq.map (fun (a,b) -> count (a,b,i))
  |> Seq.append [s |> Seq.countBy id |> Map.ofSeq |> Map.map (fun _ v -> int64 v)]
  |> Seq.reduce mergeCounts

let minMaxDiff map = map |> Map.toSeq |> Seq.map snd |> maxMin ||> (-)

charCounts rules template 10
|> minMaxDiff
|> printfn "%i"

charCounts rules template 40
|> minMaxDiff
|> printfn "%i"

2435
2587447599164
